In [11]:
import pandas as pd
from datetime import datetime as dt
import statsmodels.api as sm
import numpy as np

Step 1: Download the CRSP monthly dataset including the variables for:
1. Monthly returns including dividend distributions
2. Share Price
3. Common Shares Outstanding
4. Share code
5. CRSP permanent company identifier (Permno and or Permco)
6. Delisting return
7. Date
8. CRSP value‐weighted index returns
Keep only companies with share code < 13 (US common equities)
downloaded: check
Cleanup below

In [12]:
df = pd.read_csv('CRSP.csv')

conditional = df['SHRCD']<13

USCommondf = df[conditional].copy()

print(USCommondf.head())




C:\Users\Owner\AppData\Local\Temp\ipykernel_29596\1902099665.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('CRSP.csv')


   PERMNO        date  SHRCD  PERMCO  NWPERM DLRET      PRC        RET  \
1   10000  1986-01-31   10.0    7952     NaN   NaN -4.37500          C   
2   10000  1986-02-28   10.0    7952     NaN   NaN -3.25000  -0.257143   
3   10000  1986-03-31   10.0    7952     NaN   NaN -4.43750   0.365385   
4   10000  1986-04-30   10.0    7952     NaN   NaN -4.00000  -0.098592   
5   10000  1986-05-30   10.0    7952     NaN   NaN -3.10938  -0.222656   

   SHROUT    vwretd  
1  3680.0  0.009830  
2  3680.0  0.072501  
3  3680.0  0.053887  
4  3793.0 -0.007903  
5  3793.0  0.050847  


The sample will consist of the 500 largest firms by market capitalization (price*shares) in each of 8 time
periods give in the table below. Also, please require 36 non‐missing return observations in the three‐
year beta estimation window. For example, for the 1990 sample find the 500 largest market
capitalization firms with 36 non‐missing return observations over 1988‐1990 inclusive. You should have
a total sample of 8*500=4,000 firm‐period unit observations.

In [13]:
USCommondf.loc[:, 'date'] = pd.to_datetime(USCommondf['date'])
USCommondf.loc[:,'MCAP'] = USCommondf['PRC'] * USCommondf['SHROUT']






In [14]:
# Assuming 'USCommondf' is your DataFrame with columns 'date', 'PERMNO', 'MCAP', and others.

start_date = pd.to_datetime('1973-01-01')
end_date = pd.to_datetime('1975-12-31')

# Filter for the years 1973 to 1975.
beta75filtdf = USCommondf[(USCommondf['date'] >= start_date) & (USCommondf['date'] <= end_date)]

# Group by 'PERMNO' and filter out groups with fewer than 36 observations.
grouped_df = beta75filtdf.groupby('PERMNO').filter(lambda group: len(group) >= 36)

# Calculate the total 'MCAP' for each 'PERMNO.'
permno_mcap_totals = grouped_df.groupby('PERMNO')['MCAP'].sum().reset_index()

# Sort by total 'MCAP' in descending order and get the top 500 'PERMNO's.
top_500_permnos = permno_mcap_totals.sort_values(by='MCAP', ascending=False).head(500)

# 'top_500_permnos' now contains the 500 largest 'PERMNO's by 'MCAP' for the specified years and filter conditions.

In [15]:
# Filter 'USCommondf' for the specified date range.
filtered_75_df = USCommondf[(USCommondf['date'] >= start_date) & (USCommondf['date'] <= end_date)]

# Filter 'filtered_75_df' by the 'PERMNO's in 'top_500_permnos'.
filtered_75_df = filtered_75_df[filtered_75_df['PERMNO'].isin(top_500_permnos['PERMNO'])]

# Sort the filtered DataFrame by 'PERMNO' and 'date' to ensure data is in the correct order for calculations.
filtered_75_df = filtered_75_df.sort_values(['PERMNO', 'date'])

# Calculate 'PRC' returns from one period to the next.
filtered_75_df['prc_return'] = filtered_75_df.groupby('PERMNO')['PRC'].pct_change()

# Create an empty DataFrame to store the beta values.
beta_list = []

# Perform the regression for each 'PERMNO' to calculate the beta.
for permno, group in filtered_75_df.groupby('PERMNO'):
    X = group['vwretd']
    y = group['prc_return']
    
    model = sm.OLS(y, X, missing='drop').fit()
    beta = model.params['vwretd']
    
    beta_list.append({'PERMNO': permno, 'Beta': beta})

# Create the 'beta_75_df' DataFrame from the list of dictionaries.
beta_75_df = pd.DataFrame(beta_list)

C:\Users\Owner\AppData\Local\Temp\ipykernel_29596\379125941.py:11: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  filtered_75_df['prc_return'] = filtered_75_df.groupby('PERMNO')['PRC'].pct_change()


In [16]:
# Assuming 'beta_75_df' is your DataFrame with columns 'PERMNO' and 'Beta'.

# Sort 'beta_75_df' by 'Beta' values.
beta_75_df_sorted = beta_75_df.sort_values(by='Beta')

# Define the number of portfolios or DataFrame splits.
num_portfolios = 10
portfolio_size = len(beta_75_df_sorted) // num_portfolios

# Create a list of DataFrames to store the separate portfolios.
portfolio_dataframes = []

# Split the sorted DataFrame into ten different portfolios.
for i in range(num_portfolios):
    start_idx = i * portfolio_size
    end_idx = start_idx + portfolio_size

    # Check for the last portfolio to include the remaining rows.
    if i == num_portfolios - 1:
        portfolio = beta_75_df_sorted.iloc[start_idx:]
    else:
        portfolio = beta_75_df_sorted.iloc[start_idx:end_idx]

    portfolio_dataframes.append(portfolio)

# The 'portfolio_dataframes' list now contains ten DataFrames, each representing a different portfolio based on beta values.




In [18]:
portfolio_dataframes

[     PERMNO      Beta
 178   19940 -2.477491
 356   30672 -1.985066
 383   38578 -0.459895
 177   19916 -0.402707
 470   50171 -0.312987
 329   27318  0.109812
 343   28193  0.244089
 317   26681  0.261229
 30    11690  0.275643
 88    14656  0.324112
 304   26120  0.337645
 68    13696  0.337698
 211   21821  0.350469
 238   23210  0.408209
 261   24141  0.410503
 369   34980  0.417187
 119   16870  0.421803
 63    13522  0.427010
 166   19414  0.431702
 25    11447  0.435756
 267   24264  0.436579
 157   19043  0.443025
 372   35537  0.460177
 276   24643  0.461850
 47    12511  0.474109
 69    13821  0.492327
 233   22947  0.495240
 153   18948  0.511160
 348   28484  0.514515
 155   19019  0.518078
 247   23536  0.523409
 6     10241  0.524432
 169   19553  0.539559
 347   28425  0.542692
 435   45858  0.542862
 175   19721  0.547780
 94    14891  0.548316
 299   25785  0.552116
 201   21450  0.552175
 42    12319  0.554343
 67    13688  0.560025
 376   36679  0.560729
 379   3815

In [19]:
start_date = pd.to_datetime('1976-01-01')
end_date = pd.to_datetime('1980-12-31')

USCommon76testdf = USCommondf[(USCommondf['date'] >= start_date) & (USCommondf['date'] <= end_date)]

unique_dates = USCommon76testdf['date'].unique().tolist()

for permnos in portfolio_dataframes:
    
    






SyntaxError: incomplete input (1948597715.py, line 14)